In [2]:
import pandas as pd

import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

In [3]:
pd.read_excel("Public_Beneficiary_21_24.xlsx")

,AADHAR No,AADHAR No.1,Len,NAME,BENEFICIARY ITEM,YEAR,Category
0,9292 9645 9071,929296459071,12.0,M.KARTHIKEYAN,Push cart with Top,2021,NaN
1,2062 6807 0869,206268070869,12.0,S.RAJESHWARI,Battery and Charger,2021,NaN
2,2593 5886 7697,259358867697,12.0,M.PAECHIMUTHU,Powered Sewing Machine,2021,NaN
3,3636 6154 9968,363661549968,12.0,M.DEVI,Sewing Machine,2021,NaN
4,5263 3195 0205,526331950205,12.0,A.THIYAGARAJAN,Push cart with Top,2021,NaN
...,...,...,...,...,...,...,...
1153,NaN,345233414455,NaN,Dinesh,Gents Cycle,2024,Public
1154,NaN,363030001363,NaN,S.Anjan,Gents Cycle,2024,Public
1155,NaN,9417 6165 4055,NaN,Gajendran,Medical Aid,2024,Public
1156,NaN,382908683958,NaN,G.Sakkubhai,Tabletop Grinder,2024,Public
